In [1]:
from comet_ml import API
import comet_ml
import io
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import defaultdict
import time


from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from colour import Color



In [2]:
comet_api = API(api_key='abUSnAytqEzSzLOxNLP1ohibs')
comet_api.get()

['ketrint', 'schattengenie']

In [3]:
def get_parameter_by_key(exp, key):
    parameters = exp.get_parameters_summary()
    for parameter in parameters:
        if parameter['name'] == key:
            return parameter['valueCurrent']


In [4]:
def get_parameter_by_key(exp, key):
    parameters = exp.get_parameters_summary()
    for parameter in parameters:
        if parameter['name'] == key:
            return parameter['valueCurrent']
    return None

def new_to_old_metric(exp, key):
    metric = exp.get_metrics(key)
    vals = [float(m['metricValue']) for m in metric]
    return vals

def stack_lists(data, n=1000):
    new_data = []
    for d in data:
        if len(d) > n:
            new_data.append(d[:n])
        elif len(d) < n:
            new_data.append(
                np.concatenate([d, d[-1].repeat(n - len(d))])
            )
    return np.vstack(new_data).T

def preprocess_gp(vals):
    return np.minimum.accumulate(vals)

def add_zero_point(vals, point):
    return np.array([point] + vals.tolist())

In [5]:
experiments_names = ['val-10kcv']

In [6]:
experiments = []
for i in range(len(experiments_names)):
    experiments.append(comet_api.get(workspace='ketrint', project_name=experiments_names[i]))

In [7]:
experiments = [item for sublist in experiments for item in sublist]

In [8]:
from collections import defaultdict

good_showers = defaultdict(list)
stuck_showers = defaultdict(list)
broken_showers = defaultdict(list)
lost_showers = defaultdict(list)
x = defaultdict(list)
y = defaultdict(list)
z = defaultdict(list)
tx = defaultdict(list)
ty = defaultdict(list)

def get_data(key):
    good_showers[key].append(new_to_old_metric(exp, "Good showers")[0])
    stuck_showers[key].append(new_to_old_metric(exp, "Stuck showers"))
    broken_showers[key].append(new_to_old_metric(exp, "Broken showers"))
    lost_showers[key].append(new_to_old_metric(exp, "Lost showers"))
    x[key].append(new_to_old_metric(exp, "MAE for x"))
    y[key].append(new_to_old_metric(exp,  "MAE for y"))
    z[key].append(new_to_old_metric(exp, "MAE for z"))
    tx[key].append(new_to_old_metric(exp, "MAE for tx"))
    ty[key].append(new_to_old_metric(exp, "MAE for ty"))


In [9]:
for exp in experiments:
    if exp.get_command()[2] == './data_new/data/rand_preprocessed.pt':
        if exp.get_command()[-5] == 'True':
            key = 'HDBSCAN'
            get_data(key)
        else: 
            if exp.get_command()[14] == '5':  
                key = 'Mix Emulsion'
                get_data(key)
            elif exp.get_command()[14] == '4':
                key = 'Equal Mix'
                get_data(key)
            elif exp.get_command()[14] == '3':       
                key = 'Mix Edge'
                get_data(key)



In [10]:
dict_list = [good_showers, stuck_showers, 
                   broken_showers, lost_showers, 
                   x, y, z, tx, ty]

In [11]:
dict_names = ['good_showers', 'stuck_showers', 
                   'broken_showers', 'lost_showers', 
                   'MAE for x', 'MAE for y', 'MAE for z', 'MAE for tx', 'MAE for ty']

In [12]:
for dictionary in dict_list:
    for key in dictionary: 
        dictionary[key] = [np.round(np.mean(dictionary[key]),2), 
                           np.round(np.std(dictionary[key]), 2)]

In [13]:
df = pd.DataFrame.from_dict(dict_list)

In [14]:
df = df.assign(Metric = dict_names)

In [15]:
df = df.set_index('Metric') 

In [16]:
df

,Mix Emulsion,HDBSCAN,Equal Mix,Mix Edge
Metric,,,,
good_showers,"[0.86, 0.03]","[0.69, 0.06]","[0.86, 0.03]","[0.87, 0.02]"
stuck_showers,"[0.1, 0.04]","[0.16, 0.09]","[0.11, 0.04]","[0.1, 0.03]"
broken_showers,"[0.03, 0.01]","[0.14, 0.04]","[0.03, 0.01]","[0.03, 0.01]"
lost_showers,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]"
MAE for x,"[154.01, 9.28]","[146.81, 13.95]","[153.72, 12.13]","[154.63, 13.8]"
MAE for y,"[151.06, 10.21]","[147.01, 12.95]","[153.88, 11.45]","[156.47, 13.16]"
MAE for z,"[802.55, 50.39]","[724.76, 58.8]","[809.87, 73.17]","[823.49, 74.28]"
MAE for tx,"[0.01, 0.0]","[0.01, 0.0]","[0.01, 0.0]","[0.01, 0.0]"
MAE for ty,"[0.01, 0.0]","[0.01, 0.0]","[0.01, 0.0]","[0.01, 0.0]"


# Cross-validation performance of networks trained on different train datas

In [ ]:
experiments_names = ['clust-cross', 'test-hypo']

experiments = []
for i in range(len(experiments_names)):
    experiments.append(comet_api.get(workspace='ketrint', project_name=experiments_names[i]))
    
experiments = [item for sublist in experiments for item in sublist]

In [ ]:
len(experiments)

In [ ]:
experiments[23].get_command()

In [ ]:
for exp in experiments: 
    if exp.get_command()[2] == './data_new/data/rand_preprocessed.pt':
        if exp.get_command()[-15].split('_')[2] == '200':
            key = '200_rand'
            get_data(key)
        if exp.get_command()[-15].split('_')[2] == '50':
            key = '50_rand'
            get_data(key)           
    else:
        if exp.get_command()[-19].split('_')[2] == 'rand' and exp.get_command()[-13] == '50':  
            key = 'rand_50'
            get_data(key)
        elif exp.get_command()[-19].split('_')[2] == 'rand' and exp.get_command()[-13] == '200':
            key = 'rand_200'
            get_data(key)
        elif exp.get_command()[-19].split('_')[2] == '200' and exp.get_command()[-13] == '200':      
            key = '200_200'
            get_data(key)
        elif exp.get_command()[-19].split('_')[2] == '200' and exp.get_command()[-13] == '50':      
            key = '200_50'
            get_data(key)
        elif exp.get_command()[-19].split('_')[2] == '50' and exp.get_command()[-13] == '50':      
            key = '50_50'
            get_data(key)
        elif exp.get_command()[-19].split('_')[2] == '50' and exp.get_command()[-13] == '200':      
            key = '50_200'
            get_data(key)
        elif exp.get_command()[-19].split('_')[2] == '50' and exp.get_command()[-13] == '200':      
            key = '50_200'
            get_data(key)

In [ ]:
dict_list = [good_showers, stuck_showers, 
                   broken_showers, lost_showers, 
                   x, y, z, tx, ty]

In [ ]:
dict_names = ['good_showers', 'stuck_showers', 
                   'broken_showers', 'lost_showers', 
                   'MAE for x', 'MAE for y', 'MAE for z', 'MAE for tx', 'MAE for ty']

In [ ]:
for dictionary in dict_list:
    for key in dictionary: 
        dictionary[key] = [np.round(np.mean(dictionary[key])*100,2), 
                           np.round(np.std(dictionary[key])*100,2)]

In [ ]:
df = pd.DataFrame.from_dict(dict_list)

In [ ]:
df = df.assign(Metric = dict_names)

In [ ]:
df = df.set_index('Metric') 

In [ ]:
df